In [1]:
import csv
import os
import sys
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import time
%matplotlib inline
sys.path.append('/root/pva-faster-rcnn/lib')
sys.path.append('/root/pva-faster-rcnn/lib/datasets')
from datasets.imdb import imdb
import datasets.ds_utils as ds_utils
import numpy as np
import scipy.sparse
import scipy.io as sio
import utils.cython_bbox
import cPickle
import subprocess
import uuid
from voc_eval import voc_eval
from fast_rcnn.config import cfg
import json
from os import listdir
from os.path import isfile, join
from fast_rcnn.test import im_detect
from fast_rcnn.nms_wrapper import nms
from utils.timer import Timer
import glob
import cv2
from datasets.config import CLASS_SETS
from natsort import natsorted

In [9]:
def getAllAnnotation():
    sets = 'train'
    mapperPath = '/root/data/data-openImages_v4/class-descriptions-boxable.csv'
    classMapper = {}
    mapper,reverseMapper = loadMapper(mapperPath,classMapper)
    bboxGTPath = '/root/data/data-openImages_v4/train-annotations-bbox.csv'
    f = open(bboxGTPath, 'r')
    mappedClass = {}
    #cnt = 0
    sets = 'train'
    for row in csv.reader(f):
        #p rint row
        #cnt +=1
        if os.path.isfile(os.path.join('/root/data/data-openImages_v4/{}'.format(sets),row[0]+'.jpg')):
            if mappedClass.has_key(row[0]):
         #       print [mapper.get(row[2])]+row[3:]
                mappedClass[row[0]] += [[mapper.get(row[2])]+row[3:]]
            else:
                mappedClass[row[0]] = [[mapper.get(row[2])]+row[3:]]
        #if cnt>1000:
        #    break
    f.close()
    typeCnt = [item[0][i] for i in len(item[0]) for item in mappedClass.values()]
    types = set(typeCnt)
    inds = [typeCnt.count(i) for i in types]
    cnt = 0
    classStatistics = []
    for i in types:
        res = [i,inds[cnt]]
        #cnt +=1
        classStatistics.append(res)
    return mappedClass,classStatistics

In [3]:
def mapping(imageSizeMap,row):
    X = imageSizeMap.get(row[0])[0]
    Y = imageSizeMap.get(row[0])[1]
    x1 = float(row[4])*X+1
    y1 = float(row[5])*Y+1
    x2 = float(row[6])*X
    y2 = float(row[7])*Y
    return row[0:4]+[str(x1),str(x2),str(y1),str(y2)]+row[8:]

In [4]:
def mappedBBox():
    sets = 'test'
    bboxGTPath = '/root/data/data-openImages_v4/test-annotations-bbox.csv'
    transformedGT = '/root/data/data-openImages_v4/test-annotations-transformed.csv'
    imageSizeMap = {}
    with open("/root/pva-faster-rcnn/imgSizeMapper.csv","rb") as f:
         for row in csv.reader(f):
                target = [int(row[1].split(",")[0].split("(")[1]),int(row[1].split(",")[1].split(")")[0])]
                imageSizeMap[row[0].split(".")[0]] = target
    mappedClass = {}
    newRow = []
    #cnt = 0
    #print bboxGTPath
    f = open(bboxGTPath, 'r')
    for row in csv.reader(f):
        #print row
        #cnt +=1
        if os.path.isfile(os.path.join('/root/data/data-openImages_v4/{}'.format(sets),row[0]+'.jpg')):
            #print row
            row = mapping(imageSizeMap,row)
            #print row
            newRow.append(row)
            #break
            #if mappedClass.has_key(row[0]):
         #       print [mapper.get(row[2])]+row[3:]
            #    mappedClass[row[0]] += [[mapper.get(row[2])]+row[3:]]
            #else:
            #    mappedClass[row[0]] = [[mapper.get(row[2])]+row[3:]]
        #if cnt>1000:
        #    break
    f.close()
    with open(transformedGT,'wb') as f:
        writer = csv.writer(f)
        writer.writerows(newRow)
    return newRow

In [79]:
#typeCnt = [item[i][0] for i in range(len(item)) for item in mappedClass.values()]

typeCnt = []
imgCnt = {}
typeFreq = []
total = 0
for i,j in mappedClass.iteritems():
    imgName = i
    item = []
    for k in range(len(j)):
        label = j[k][0]
        total +=1
        if label not in item:
            typeFreq.append(label)
            item.append(label)


freq = set(typeFreq)
freqInds = [typeFreq.count(i) for i in freq]
cnt = 0
freqStatistics = []
for i in freq:
    res = [i,freqInds[cnt]]
    cnt +=1
    freqStatistics.append(res)


In [80]:
print freqStatistics

[['Snack', 10313], ['Van', 5378], ['Waffle iron', 23], ['Accordion', 839], ['Gondola', 569], ['Elephant', 1732], ['Baked goods', 10289], ['Axe', 115], ['Porcupine', 203], ['Plate', 2023], ['Umbrella', 3357], ['Ipod', 414], ['Harmonica', 20], ['Watch', 1434], ['Polar bear', 490], ['Rifle', 1752], ['Goldfish', 407], ['Organ', 377], ['Antelope', 848], ['Candy', 648], ['Chopsticks', 470], ['Wine glass', 4894], ['Barrel', 662], ['Picture frame', 5536], ['Home appliance', 1051], ['Hamburger', 1068], ['Filing cabinet', 275], ['Canoe', 2076], ['Sports uniform', 6531], ['Lifejacket', 1222], ['Canary', 231], ['Juice', 1746], ['Motorcycle', 6944], ['Can opener', 6], ['Panda', 574], ['Turkey', 367], ['Willow', 193], ['Camera', 5037], ['Suit', 43922], ['Orange', 900], ['Jet ski', 458], ['Computer keyboard', 3331], ['Fork', 1034], ['Briefcase', 111], ['Ski', 1361], ['Squash', 122], ['Chainsaw', 119], ['Wardrobe', 208], ['Doughnut', 400], ['Digital clock', 169], ['Golf ball', 321], ['Spice rack', 66]

In [62]:
cnt = 0
classStatistics = []
for i in types:
    res = [i,inds[cnt]]
    cnt +=1
    classStatistics.append(res)
print classStatistics

[['Snack', 37374], ['Van', 7720], ['Waffle iron', 31], ['Accordion', 955], ['Gondola', 1868], ['Elephant', 3272], ['Baked goods', 23010], ['Axe', 148], ['Porcupine', 229], ['Plate', 5416], ['Umbrella', 7204], ['Ipod', 595], ['Harmonica', 38], ['Watch', 1903], ['Polar bear', 664], ['Rifle', 2540], ['Goldfish', 2204], ['Organ', 398], ['Antelope', 1568], ['Candy', 2261], ['Chopsticks', 617], ['Wine glass', 12934], ['Barrel', 2086], ['Picture frame', 11957], ['Home appliance', 2086], ['Hamburger', 1486], ['Filing cabinet', 374], ['Canoe', 4543], ['Sports uniform', 19396], ['Lifejacket', 3678], ['Canary', 387], ['Juice', 2838], ['Motorcycle', 13382], ['Can opener', 7], ['Panda', 882], ['Turkey', 734], ['Willow', 735], ['Camera', 6404], ['Suit', 110848], ['Orange', 6195], ['Jet ski', 543], ['Computer keyboard', 4542], ['Fork', 1687], ['Briefcase', 162], ['Ski', 3505], ['Squash', 375], ['Chainsaw', 130], ['Wardrobe', 238], ['Doughnut', 930], ['Digital clock', 199], ['Golf ball', 434], ['Spice

In [63]:
imageSizeMap = {}
with open("/root/pva-faster-rcnn/imgSizeMapper.csv","rb") as f:
     for row in csv.reader(f):
            target = [int(row[1].split(",")[0].split("(")[1]),int(row[1].split(",")[1].split(")")[0])]
            imageSizeMap[row[0].split(".")[0]] = target

mappedClass,classStatistics = getAllAnnotation()

KeyboardInterrupt: 

In [ ]:
labelList = ['Backpack','Baseball bat','Baseball glove', 'Bag','Man','Woman',
             'Bicycle','Rifle','Handgun','Shotgun','Suit','Gun',
             'Dress','Jacket','Skirt','Shorts','Sports Uniform','Shirt','Trousers',
             'Mini skirt','Goggles','Glasses','Headphones','Sunglasses',
             'Hat','Helmet','Sun hat','Football helmet','Cowboy hat','Sombrero','Roller skates',
             'Boot','Handbag','Backpack','Suitcase','Plastic bag','Briefcase','Wheelchair',
             'Umbrella','Scarf','Tie']


mapperList = {'Rifle':'Gun','Handgun':'Gun','Shotgun':'Gun','Boy':'Man','Girl':'Woman',
              'Mini skirt':'Skirt','Goggles':'Glasses','Sunglasses':'Glasses','Sun hat':'Hat',
              'Football helmet':'Helmet','Cowboy hat':'Hat','Sombrero':'Hat','Handbag':'Bag',
              'Plastic bag':'Bag','Briefcase':'Bag'}


animalList= ['Pig','Monkey','Bull','Deer','Bear']

knife = ['Dagger','Sword',]

In [7]:
def loadMapper(mapperPath,mapperList):
    mapper = {}
    reverseMapper = {}
    f = open(mapperPath,'r')
    for i in csv.reader(f):
        key = i[0]
        if mapperList.has_key(i[1]):
            val = mapperList.get(i[1])
        else:
            val = i[1]
        mapper[key] = val
        reverseMapper[val] = key
    f.close()
    return mapper,reverseMapper  


def getAllAnnotation():
    mapperPath = '/root/data/data-openImages_v4/class-descriptions-boxable.csv'
    classMapper = {}
    mapper,reverseMapper = loadMapper(mapperPath,classMapper)
    bboxGTPath = '/root/data/data-openImages_v4/{}-annotations-bbox.csv'.format(sets)
    f = open(bboxGTPath, 'r')
    annotations = {}
    mappedClass = {}
    for row in csv.reader(f):
        if os.path.isfile(os.path.join('/root/data/data-openImages_v4/{}'.format(sets),row[0]+'.jpg')):
            if annotations.has_key(row[0]):
                mappedClass[row[0]] += [[mapper.get(row[2])]+row[3:]]
            else:
                mappedClass[row[0]] = [[mapper.get(row[2])]+row[3:]]
    f.close()
    typeCnt = [item[0][0] for item in mappedClass.values()]
    types = set(typeCnt)
    inds = [typeCnt.count(i) for i in types]
    cnt = 0
    classStatistics = []
    for i in types:
        res = [i,inds[cnt]]
        cnt +=1
        classStatistics.append(res)
    return mappedClass,classStatistics

def getAnnotation(labelList=None,mapperList=None,sets='train',mannual=True,classify=True):
    tStart = time.time()
    classMapper = {}
    if mapperList != None:
        classMapper = mapperList
        print len(classMapper)
    else:
        for i in labelList:
            classMapper[i] = i 
    mapperPath = '/root/data/data-openImages_v4/class-descriptions-boxable.csv'
    mapper,reverseMapper = loadMapper(mapperPath,classMapper)
    print len(mapper)
    print len(reverseMapper)
    #print mapper
    if mannual:
        method = ['activemil','freeform ','xclick']
    else:
        method = ['freeform ','xclick']
    bboxGTPath = '/root/data/data-openImages_v4/{}-annotations-bbox.csv'.format(sets)
    seq = [reverseMapper.get(i) for i in labelList]
    f = open(bboxGTPath, 'r')
    annotations = {}
    mappedClass = {}
    for row in csv.reader(f):
        if row[1] not in method:
            continue
        #if row[2] not in seq:
        #    continue
        if os.path.isfile(os.path.join('/root/data/data-openImages_v4/{}'.format(sets),row[0]+'.jpg')):
            if annotations.has_key(row[0]):
                annotations[row[0]] += [row[2:]]
                mappedClass[row[0]] += [[mapper.get(row[2])]+row[3:]]
            else:
                annotations[row[0]] = [row[2:]]
                mappedClass[row[0]] = [[mapper.get(row[2])]+row[3:]]
    f.close()
    tEnd = time.time()
    print "It cost %f sec" % (tEnd - tStart)
    return [annotations,mappedClass]


def plotClassDistributionBar(annotations):
    typeCnt = [item[0] for item in annotations.values()]
    types = set(typeCnt)
    inds = [typeCnt.count(i) for i in types]
    plt.rcParams.update({'font.size': 16})
    fig, ax = plt.subplots()    
    fig.set_size_inches(7.5, 10, forward=True)
    width = 0.75 # the width of the bars 
    ind = np.arange(len(inds))  # the x locations for the groups
    ax.barh(ind, inds, width, color="blue")
    ax.set_yticks(ind+width/2)
    ax.set_yticklabels(types, minor=False)
    plt.title('total data points:{}'.format(len(annotations)))
    plt.xlabel('Counts')
    plt.ylabel('Total : {} Classes'.format(len(types)))
    plt.rcParams.update({'font.size': 12})
    for i, v in enumerate(inds):
        ax.text(v + 100, i + .25, str(v), color='green', fontweight='bold',)

def plotClassDistributionPie(annotations):
    typeCnt = [item[0] for item in annotations.values()]
    types = set(typeCnt)
    inds = [typeCnt.count(i) for i in types]
    cs=cm.Set1(np.arange(40)/40.)
    plt.rcParams.update({'font.size': 12})
    fig, ax = plt.subplots()  
    fig.set_size_inches(10.5, 10.5, forward=True)
    ax.pie(inds,labels=types,autopct='%.1f%%',colors=cs);
    

def dumpJson(datasetName,annotation):
    imgSizeMapPath = '/root/pva-faster-rcnn/imgSizeMapper.csv'
    f = open(imgSizeMapPath, 'r')
    imgSizeMap = {}
    for row in csv.reader(f):
        imgSizeMap[row[0]]=row[1]
    f.close()
    vaticForm = {}
    vaticJson = dict()
    for key,labels in annotation.iteritems():
        cnt = 0
        targetDict = {}
        for label in labels:   
            cnt +=1 
            if label[8]!='1':
                imgPath = key+'.jpg'
                imgSize = imgSizeMap.get(imgPath)
                y = float(imgSize.split(",")[0][1:])
                x = float(imgSize.split(",")[1].split(")")[0])
                bbox = [float(i)*x for i in label[2:4]] + [float(i)*y for i in label[4:6]]
                vaticForm = label[0:2]+bbox+label[6:]
                target = {
                    u'x1':int(vaticForm[2]),
                    u'height':int(vaticForm[5]-vaticForm[4]),
                    u'attribute':[],
                    u'y1':int(vaticForm[4]),
                    u'width':int(vaticForm[3]-vaticForm[2]),
                    u'label':vaticForm[0],
                    u'occluded':int(vaticForm[6])        
                }
                targetDict[str(cnt)]=target
        vaticJson[key] = targetDict

    with open(datasetName+'.json', "w") as f:
        json.dump(vaticJson,f)

In [ ]:
vaticForm = {}
vaticJson = dict()
datasetName='25typesTrain'
for key,labels in annotations[1].iteritems():
    cnt = 0
    targetDict = {}
    for label in labels:   
        cnt +=1 
        if label[8]!='1':
            imgPath = key+'.jpg'
            imgSize = imgSizeMap.get(imgPath)
            y = float(imgSize.split(",")[0][1:])
            x = float(imgSize.split(",")[1].split(")")[0])
            bbox = [float(i)*x for i in label[2:4]] + [float(i)*y for i in label[4:6]]
            vaticForm = label[0:2]+bbox+label[6:]
            target = {
                u'x1':int(vaticForm[2]),
                u'height':int(vaticForm[5]-vaticForm[4]),
                u'attribute':[],
                u'y1':int(vaticForm[4]),
                u'width':int(vaticForm[3]-vaticForm[2]),
                u'label':vaticForm[0],
                u'occluded':int(vaticForm[6])        
            }
            targetDict[str(cnt)]=target
    vaticJson[key] = targetDict

    
print len(vaticJson)     


In [ ]:
with open(datasetName+'.json', "w") as f:
    json.dump(vaticJson,f)

In [ ]:
sets = 'train'
#annotations = getAnnotation(labelList,mapperList,sets,False,True)
annotations,classStatistics = getAllAnnotation()

In [ ]:
dumpJson('Alltypes',annotations[1])

In [ ]:
import json
dataset = 'openImages_v4'
#dataset = 'airport'
jsonInput = open('/root/data/data-{}/annotations.json'.format(dataset),'r')

#jsonInput = open('25typesTrain.json','r')
annos = json.load(jsonInput)
frames = {}
for key,value in annos.iteritems():
    print key, value
    break

In [ ]:
import json
dataset = 'openImages_v4'
dataset = 'airport'
jsonInput = open('/root/data/data-{}/annotations.json'.format(dataset),'r')

#jsonInput = open('25typesTrain.json','r')
annos = json.load(jsonInput)
frames = {}
for key,value in annos.iteritems():
    print key, value
    break

In [ ]:
print len(annotations[0])

In [81]:
with open("imgStatistic.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(freqStatistics)

In [ ]:
def get_data_map(path="/root/data", prefix="data-"):
    data_map = {}
    data_paths = glob.glob("{}/{}*".format(path, prefix))
    for data_path in data_paths:
        #print data_path
        name = os.path.basename(data_path)[5:]
        data_map[name] = data_path
    return data_map

def load_meta(meta_path):
    if os.path.exists(meta_path):
        meta = json.load(open(meta_path))
    else:
       
        meta = {"format":"jpg"}
        meta["train"] = {"start":None, "end":None, "stride":1, "sets":[0]}
        meta["test"] = {"start":None, "end":None, "stride":30, "sets":[1]}
        print("Meta data path: {} does not exist. Use Default meta data".format(meta_path))
    return meta


data_map = get_data_map()
data_names = data_map.keys()


In [ ]:
data_map['airport']